In [1]:
from typing import Any, Dict, List

from kerbal_api.querying import execute_query, get_default_adapter

adapter = get_default_adapter()

In [2]:
def run_query(query: str, args: Dict[str, Any]) -> List[Dict[str, Any]]:
    return list(execute_query(adapter, query, args))

In [3]:
query = """
{
    Part {
        name @output(out_name: "part_name")
        cost @output(out_name: "part_cost")
        dry_mass @output(out_name: "part_dry_mass")
        
        out_Part_EngineModule {
            throttleable @output(out_name: "throttleable")
            isp_vacuum @output(out_name: "isp_vacuum")
        }
    }
}
"""
args = {}

In [4]:
query = """
{
    Part {
        name @output(out_name: "part_name")
        cfg_file_path @output(out_name: "part_cfg_file")
        cost @output(out_name: "part_cost")
        dry_mass @output(out_name: "part_dry_mass")
        
        out_Part_EngineModule {
            throttleable @filter(op_name: "=", value: ["$throttleable"])
            isp_vacuum @output(out_name: "isp_vacuum")
        }
    }
}
"""
args = {
    "throttleable": False,
}

In [5]:
query = """
{
    Part {
        name @output(out_name: "part_name")
        cfg_file_path @output(out_name: "part_cfg_file")
        cost @output(out_name: "part_cost")
        dry_mass @output(out_name: "part_dry_mass")
        
        out_Part_EngineModule {
            isp_at_1atm @filter(op_name: "is_null")
        }
    }
}
"""
args = {}

In [6]:
query = """
{
    Part {
        internal_name @filter(op_name: "=", value: ["$internal_name"])
        cost @output(out_name: "part_cost")
        dry_mass @output(out_name: "part_dry_mass")
        
        out_Part_EngineModule {
            min_thrust @output(out_name: "min_thrust")
            max_thrust @output(out_name: "max_thrust")
            throttleable @output(out_name: "throttleable")
            isp_vacuum @output(out_name: "isp_vacuum")
            isp_at_1atm @output(out_name: "isp_at_1atm")
        }
    }
}
"""
args = {
    "internal_name": "RAPIER",
}

In [7]:
run_query(query, args)

[{'part_cost': 6000,
  'part_dry_mass': 2.0,
  'min_thrust': 0.0,
  'max_thrust': 105.0,
  'throttleable': True,
  'isp_vacuum': 3200.0,
  'isp_at_1atm': None},
 {'part_cost': 6000,
  'part_dry_mass': 2.0,
  'min_thrust': 0.0,
  'max_thrust': 180.0,
  'throttleable': True,
  'isp_vacuum': 305.0,
  'isp_at_1atm': 275.0}]